In [1]:
from whitespace_correction import WhitespaceCorrector
from tqdm.notebook import tqdm

In [2]:
cor = WhitespaceCorrector.from_pretrained(model="eo_larger_byte", device="cuda:0", download_dir="/home/css/models/wsc")

2024-11-26 09:22:29,979 [WHITESPACE CORRECTION DOWNLOAD] [INFO] eo_larger_byte is already downloaded to download directory /home/css/models/wsc
/home/liujunhui/miniconda3/envs/semantic_uncertainty/lib/python3.11/site-packages/text_utils/io.py:74: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have

In [3]:
## 文件格式转换
import json

data_path_1 = "/home/liujunhui/workspace/proj/WSC/dataset/jfleg_txt/jfleg/sources.txt"
data_path_2 = "/home/liujunhui/workspace/proj/WSC/dataset/jfleg_txt/jfleg/corrections.txt"
# 打开文件并读取内容到列表
with open(data_path_1, "r") as file:
    lst_1 = [line.strip() for line in file.readlines()]

with open(data_path_2, "r") as file:
    lst_2 = [line.strip() for line in file.readlines()]

dic = {
    "original":lst_1 + lst_2
}

# 转换文件格式为json,保存为json格式
with open("dataset/jfleg.json", "w", encoding="utf-8") as f:
    json.dump(dic, f, ensure_ascii=False, indent=4)

## 读取json文件，获取original 文本，进行wsc纠正

In [4]:
with open("dataset/jfleg.json", "r", encoding="utf-8") as f:
    data = json.load(f)

lst_original = data['original']

lst_wsc = []
for item in tqdm(lst_original):
    corrected_string = cor.correct_text(item)
    lst_wsc.append(corrected_string)


# 保存wsc后的数据
dic = {
    "original":lst_original,
    "wsc":lst_wsc
}
with open("dataset/jfleg.json", "w", encoding="utf-8") as f:
    json.dump(dic, f, ensure_ascii=False, indent=4)

# 单独保存下游任务需要的txt文件
mid = len(lst_wsc) // 2
wsc_1=lst_wsc[:mid]
wsc_2=lst_wsc[mid:]
with open("dataset/jfleg_txt/jfleg_corrected/sources.txt", "w") as file:
    for i, line in enumerate(wsc_1):
        if i < len(wsc_1) - 1:
            file.write(line + "\n")
        else:
            file.write(line)

with open("dataset/jfleg_txt/jfleg_corrected/corrections.txt", "w") as file:
    for i, line in enumerate(wsc_2):
        if i < len(wsc_2) - 1:
            file.write(line + "\n")
        else:
            file.write(line)

  0%|          | 0/3202 [00:00<?, ?it/s]

/home/liujunhui/miniconda3/envs/semantic_uncertainty/lib/python3.11/site-packages/text_utils/modules/grouping.py:36: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(enabled=False):
